In [ ]:
import os
img_dir = '/tmp/nst/'
if not os.path.exists(img_dir):
    os.makedirs(img_dir)
!wget --quiet -P /tmp/nst/ https://upload.wikimedia.org/wikipedia/commons/d/d7/Green_Sea_Turtle_grazing_seagrass.jpg
!wget --quiet -P /tmp/nst/ https://upload.wikimedia.org/wikipedia/commons/0/0a/The_Great_Wave_off_Kanagawa.jpg
!wget --quiet -P /tmp/nst/ https://upload.wikimedia.org/wikipedia/commons/b/b4/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg
!wget --quiet -P /tmp/nst/ https://upload.wikimedia.org/wikipedia/commons/0/00/Tuebingen_Neckarfront.jpg
!wget --quiet -P /tmp/nst/ https://upload.wikimedia.org/wikipedia/commons/6/68/Pillars_of_creation_2014_HST_WFC3-UVIS_full-res_denoised.jpg
!wget --quiet -P /tmp/nst/ https://upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg/1024px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg

from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import ipywidgets as widgets
#import cv2
from scipy.signal import convolve
from ipywidgets import interact, fixed
content_path = '/tmp/nst/Green_Sea_Turtle_grazing_seagrass.jpg'
style_path = '/tmp/nst/The_Great_Wave_off_Kanagawa.jpg'
cube_path = 'cube.png'
wheel_path = 'wheel.jpeg'
turtle = mpimg.imread(content_path)
wave = mpimg.imread(style_path)
cube = mpimg.imread(cube_path) * 255
wheel = mpimg.imread(wheel_path)
white = mpimg.imread("white.jpg")
black = mpimg.imread("black.png")
def make_black_and_white(image):
  return image @ (1/3,1/3,1/3)

def show_image(image):
  if image.ndim == 2:
    _ = plt.imshow(image.astype('uint8'),cmap='Greys')
  else:
    _ = plt.imshow(image.astype('uint8'))

  
black_and_white_turtle = make_black_and_white(turtle).astype('uint8')

def blur(image,size):
  if len(image.shape) == 3:
    return blur3d(image,size)
  C = np.ones([size,size]) / (size**2)
  return convolve(image,C)


def blur3d(image,size):
  A = []
  for i in range(3):
    channel = image[:,:,i]
    A.append(blur(channel,size))
  return np.dstack(A)

def make_slider():
    return widgets.FloatSlider(value=.5,min=0,max=1.0,step=.1)

def channel(image,x,y,z):
    return image * (x,y,z)

def plot_channel(image,red,green,blue):
    show_image(channel(image,red,green,blue))

def interactive_channel(image):
    interact(plot_channel,image = widgets.fixed(image), red = make_slider(),green=make_slider(),blue=make_slider())
    
def detect_edges(image):
    if image.ndim == 3:
        image = make_black_and_white(image)
    
def separate_image(image):
    _=fig, ax = plt.subplots(2, 2, sharex='col', sharey='row',figsize=[8,8])
    _=ax[0][0].imshow(image  * (1,1,1))
    _=ax[0][1].imshow(image  * (1,0,0))
    _=ax[1][0].imshow(image * (0,1,0))
    _=ax[1][1].imshow(image * (0,0,1))

Ignore everything before this line, it's not important to the lesson, but is needed to make everything else work. Before starting, go to the tab above that says **Runtime**, and then click on **Restart and Run All**. Please let me know if you have trouble.



# Matrices in Real Life

Matrices are a main way that computers store data. In this lesson, we will learn how computers use matrices to store images. Do not worry about how the code works, we are mainly interested in the math behind it! Feel free to ask me after class if you want to learn more about it.


# Black and White Images

A black and white image is just a matrix of values between 0-255.  Let $A$ be the image. $A_{ij}$ represents the intensity of the image at point $i,j$. 0 means black, 255 means white, and everything in between is gray.

This means the image matrix has 2525 rows and 3367 columns.

In [ ]:
black_and_white_turtle

In [ ]:
show_image(black_and_white_turtle)


# Color 

But black and white images are boring! Instead of a matrix of numbers, color image is a matrix of length 3 vectors. $C_{ij} = (r,g,b)$ In fact, a color image can be separated into red, green, and blue components. Let's explore how!

In [ ]:
separate_image(turtle)

In [ ]:
turtle.shape

The color version of turtle has 2525 rows, 3367 columns, and 3 channels (one for each color).

We can multiply all elements in each matrix in image by a number between 0-1.

Let $v = (x,y,z)$.

$C * v = (R*x, G*y, B*z)$

The shape does not change, just the values. 

If $v=(1,1,1)$ we get just the original image. What do you think would happen if we change $v$?

In [ ]:
x,y,z = 1,1,1
v = x,y,z
show_image(turtle * v)

Let's try making $y$ and $z$ smaller while keeping $x$ the same.

In [ ]:
x,y,z=1,.5,.5
v=x,y,z
show_image(turtle*v)


The image is now more red! What do you think would happen if we make one color 1, and the others 0?

In [ ]:
x,y,z = 0,0,1
v = x,y,z
show_image(wave * v)

The image is now completely blue! Now experiment with changing $x,y,z$!

In [ ]:
x,y,z = .9,2,.4
v = x,y,z
show_image(wave * v)

In [ ]:
interactive_channel(wheel)

# Make a Color Image Black and White

A color image can be converted into a black and white one. To do this, we will multiply the matrix by the vector (1/3,1/3,1/3). This just takes the average value of the 3 channels. Note: 

```
@
``` means matrix multiplication in the code.

Let $B$ be the Black and White version of image $A$.
$$v = (1/3,1/3,1/3)$$
$$\implies B = C v^T = (1/3R + 1/3G + 1/3B)$$


In [ ]:
black_and_white_wave = wave @ (1/3,1/3,1/3)

In [ ]:
show_image(black_and_white_wave)

# Negative Images

To make an image negative, we will want to reverse the value of each pixel.

Let $N$ be the negative matrix. $N_{ijk} = 255 - A_{ijk}$.

In [ ]:
show_image(255 - cube)

# For Fun: More Advanced Image Filters

It's ok if you don't completely understand the math behind these filters, they are more complicated than the ones we have used so far.

# Blurry Images
To blur an image, we convolve a with another matrix. What this does is take the average of the pixels around it.


It is hard to tell that a pixel is blurred at all when the size of the convolution matrix is only 10x10.

In [ ]:
blurry_image = blur(image=turtle,size=10)
show_image(blurry_image)

When increased to 100x100, the image is very blurry.

In [ ]:
blurry_image = blur(image=wave,size=160)
show_image(blurry_image)

In [ ]:
from scipy.signal import convolve2d

In [ ]:
edged=np.clip(convolve(cube@(1/3,1/3,1/3),edge_matrix),a_min=0,a_max=255)

In [ ]:
edged=np.clip(convolve(cube@(1/3,1/3,1/3),edge_matrix),a_min=0,a_max=255)

In [ ]:
show_image(edged)

# Why this matters

Image processing techniques are of growing importance today. The field of using computers to understand image data is called Computer Vision. It is used in technologies including:


*   Self driving cars
*   Face Recognition
*   Cancer detection
*   Product Sorting
*   Robotics

A fun example of Computer Vision in action:

https://www.youtube.com/watch?v=AboxUKxRYJw



In [ ]:
%matplotlib notebook

import cv2
import matplotlib.pyplot as plt
import signal

In [ ]:
def signal_handler(signal, frame):
    # KeyboardInterrupt detected, exiting
    global is_interrupted
    is_interrupted = True
    

vc = cv2.VideoCapture(0)

plt.ion()

if vc.isOpened(): # try to get the first frame
    is_capturing, frame = vc.read()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)    # makes the blues image look real colored
    webcam_preview = plt.imshow(frame)    
else:
    is_capturing = False

signal.signal(signal.SIGINT, signal_handler)
is_interrupted = False
while is_capturing:
    is_capturing, frame = vc.read()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)    # makes the blues image look real colored
    webcam_preview.set_data(frame)
    plt.draw()

    try:    # Avoids a NotImplementedError caused by `plt.pause`
        plt.pause(0.05)
    except Exception:
        pass
    if is_interrupted:
        vc.release()
        break    
    

In [ ]:
import cv2
import sys

cascPath = sys.argv[1]
faceCascade = cv2.CascadeClassifier(cascPath)

video_capture = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30),
        flags=cv2.CASCADE_SCALE_IMAGE
    )

    # Draw a rectangle around the faces
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    # Display the resulting frame
    cv2.imshow('Video', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything is done, release the capture
video_capture.release()
cv2.destroyAllWindows()

In [ ]:
cv2.CASCADE_SCALE_IMAGE